In [1]:
# imports and defaults

import importlib
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import prettyplot as pp
import cn_derivations as cn
import numpy as np
import os
import data_parser as dp
import math
import mpmath
from timeit import default_timer as timer
import r_function_port as rfp
from sklearn.linear_model import LinearRegression
mpmath.dps = 30
mpmath.pretty = True

pd.options.display.max_rows = 10
import seaborn as sns
sns.set_style('ticks')
sns.set_context('talk')

# Time zone adjustment
import matplotlib
matplotlib.rcParams['timezone'] = 'CET'
os.getcwd()


'/home/nick/PycharmProjects/bkp/ss19-feldkurs/Pycode'

In [2]:
"""Data overview"""
filename = "2019-05-23"
station = "hungerburg"

hsdf = dp.scintillometer_parse(filename)
# sdf["Cn2"] = (sdf["Cn2"]) / 2032**(-3) * 1031.5**(-3)
# sdf["H_convection"] = sdf["H_convection"] / 2032**(-3) * 1031.5**(-3)
importlib.reload(cn)
hcomputed_data = cn.data_processor(filename, station)
hcdf = hcomputed_data["computed"]
hz_eff = hcomputed_data["effective_height"]
hcdf = hcdf.loc[:"2019-05-23 07:55"].copy(deep=True)
hsdf = hsdf.loc[:"2019-05-23 07:55"].copy(deep=True)
ahcdf = hcdf.loc[:"2019-05-23 05:57"]
hcdf.index = hcdf.index.time

Please enter the stability conditions (stable, unstable, or other):
u
Mean path height: 84.24445786885246m
Effective path height: 47.645469666029996m


In [3]:
ahcdf = hcomputed_data["computed"].copy(deep=True)
ahcdf = ahcdf.loc[:"2019-05-23 05:57"]
ahcdf.index = ahcdf.index.time

In [4]:
"""Ward's iteration"""
importlib.reload(rfp)

# Takes 100s or so to run
hcdf_backup = hcdf.copy(deep=True)  # in case of later testing

switch_time = "0557"
computed_data_stable = cn.data_processor("2019-05-23", "hungerburg")

z_eff_s = computed_data["effective_height"]
hcdf_stable = hcdf.copy(deep=True)
# hcdf_stable = computed_data_stable["computed"]

scdf

Please enter the stability conditions (stable, unstable, or other):
s
Mean path height: 84.24445786885246m
Effective path height: 78.48718364377896m


NameError: name 'computed_data' is not defined

In [ ]:

header_list = ["CT2", "Cn2","H_free","Q_0","l_ob","obukhov","obukhov_diff", "pressure","rho_air",
               "shf","shf_diff","temperature", "theta_star","u_star", "windspeed"]
scdf = pd.read_csv("../data/weather_data/schiessstand_bound.csv",
                           header=None, skiprows=1, index_col=0,
                           sep=",", names=header_list, parse_dates=True)
scdf = scdf.loc[:"2019-05-24 07:55"].copy(deep=True)
ascdf = scdf.loc[:"2019-05-24 05:57"]
scdf.index = scdf.index.time
ascdf.index = ascdf.index.time


In [ ]:
# """Data overview"""
# filename = "2019-05-24"
# station = "schiessstand"

# ssdf = dp.scintillometer_parse(filename)
# ssdf["Cn2"] = (ssdf["Cn2"]) / 2032**(-3) * 1031.5**(-3)
# ssdf["H_convection"] = ssdf["H_convection"] / 2032**(-3) * 1031.5**(-3)
# importlib.reload(cn)
# scomputed_data = cn.data_processor(filename, station)
# scdf = scomputed_data["computed"]
# sz_eff = scomputed_data["effective_height"]
# scdf = scdf.loc[:"2019-05-24 07:55"].copy(deep=True)
# ssdf = ssdf.loc[:"2019-05-24 07:55"].copy(deep=True)

In [ ]:
fig = plt.figure(figsize=(26,8))

ascdf["CT2"].plot(color="black", label="Schießstand")
ahcdf["CT2"][ahcdf["CT2"]<0.008].plot(color="black", linestyle="dashed", label="Hungerburg")
ax = plt.gca()
# ax.xaxis.set_major_formatter(dates.DateFormatter('%H:%M'))  # hours and minutes
plt.title("Structure Parameter of the Refractive Index of Temperature, C$_{T}^{2}$", fontsize=24, fontweight="bold")
plt.ylabel("Structure Parameter, [K$^{2}m^{-{2}/{3}}$]", fontsize=20)
plt.xlabel("Time, CET")
plt.legend()
plt.show()
fig.savefig('./plots/ZAMG/CT2_comp.png', bbox_inches='tight')

In [ ]:
fig = pp.plot_generic(cdf, "CT2")
plt.show()

In [ ]:
"""Ward's iteration"""
importlib.reload(rfp)

# Takes 100s or so to run
cdf_backup = cdf.copy(deep=True)  # in case of later testing

switch_time = "0557"
computed_data_stable = cn.data_processor(filename, station)

z_eff_s = computed_data["effective_height"]
cdf_stable = computed_data_stable["computed"].iloc[
    cdf.index.indexer_between_time("00:00", switch_time)].copy(deep=True)

cdf_stable = rfp.ward_method(cdf_stable, z_eff, "stable")


In [ ]:
cdf_stable

In [ ]:
switch_time = "0557"
computed_data_unstable = cn.data_processor(filename, station)
z_eff_u = computed_data["effective_height"]

cdf_unstable = computed_data_unstable["computed"].iloc[
    cdf.index.indexer_between_time(switch_time, "23:59")].copy(deep=True)

cdf_unstable = rfp.ward_method(cdf_unstable, z_eff_u, "unstable")

In [ ]:
cdf_stable["shf"] = -cdf_stable["shf"]
cdf_stable["obukhov"] = -cdf_stable["obukhov"]

cdf = cdf_stable.append(cdf_unstable,sort=True)

In [ ]:
cdf

In [ ]:
import importlib
importlib.reload(pp)
fig = pp.plot_generic(cdf, "shf")
plt.show()

In [ ]:
header_list = ["year", "month", "day", "hour", "minute", "second", "shf", "windspeed", "obukhov"]
rooftop = pd.read_csv("../../data/weather_data/rooftop.csv",
                           header=None, skiprows=1, index_col=None,
                           names=header_list, sep=",")

t_cols = ['year', 'month', 'day', 'hour', 'minute', "second"]
rooftop.index = pd.to_datetime(rooftop[t_cols])

# Remove the useless columns
rooftop = rooftop.drop(t_cols, axis=1)
rooftop = rooftop.replace(-999, np.nan)
rooftop = rooftop.fillna(method="ffill")

oidx = rooftop.index
nidx = pd.date_range(oidx.min(), oidx.max(), freq='60s')
rooftop = rooftop.reindex(oidx.union(nidx)).interpolate(
    'index').reindex(nidx).tz_localize("UTC")
rooftop.index = rooftop.index + pd.DateOffset(hours=1)
rooftop

In [ ]:
wdf = dp.weather_download("2019-05-24")
wdf = wdf.loc["2019-05-24 03:30":"2019-05-24 15:00"]
wdf

In [ ]:
hourly_cdf = pd.DataFrame()
hourly_cdf["windspeed_mean"] = 0
# hourly_cdf["windspeed_std"] = hourly_cdf["windspeed"].astype("float64").resample("H").std()
hourly_cdf["windspeed_mean"] = cdf["windspeed"].astype("float64").resample("600S").mean()

hourly_roof = pd.DataFrame()
hourly_roof["windspeed_mean"] = 0
# hourly_cdf["windspeed_std"] = hourly_cdf["windspeed"].astype("float64").resample("H").std()
hourly_roof["windspeed_mean"] = rooftop["windspeed"].astype("float64").resample("600S").mean()

fig = plt.figure(figsize=(26, 8))
(cdf["windspeed"]).astype("float64").plot(color="grey", label="Scintillometer")
# hourly_cdf["windspeed_mean"].astype("float64").plot(
#     color="black", linestyle="dashed", label="Scintillometer, hourly mean")

(rooftop["windspeed"]).astype("float64").plot(color="red", linestyle = "dashed", label="InnFLUX, half-hourly mean")
wdf["wg"].astype("float64").plot(color="black", linestyle="dashed", label="ZAMG Hourly Mean")

plt.legend(loc="upper left", fontsize = 20)
plt.xlabel("Time, CET", fontsize = 20)
# plt.ylabel("Mean Wind Speeds, [m$\cdot$s$^{-2}$]", fontsize = 20)
# plt.ylabel("Obukhov Length, [m]", fontsize=20)
# plt.title("Obukhov Lengths for Schießstand, 24$^{th}$ May 2019", fontweight="bold", fontsize = 24)
plt.title("Mean Wind Speeds for Schießstand, 24$^{th}$ May 2019", fontweight="bold", fontsize = 24)
plt.ylabel("Mean Wind Speed, [m$\cdot s^{-2}$]", fontsize=20)
ax = plt.gca()
ax.xaxis.set_major_formatter(
    dates.DateFormatter('%H:%M'))  # hours and minutes
plt.show()

# fig.savefig('./plots/ERTEL/windspeed_series.png', bbox_inches='tight')

In [ ]:
cdf["shf"].iloc[4:]

In [ ]:
rooftop["shf"].iloc[:611]

In [ ]:
df = pd.DataFrame()
# df = cdf.filter([""])
df["shf"] = cdf["shf"].iloc[4:].astype("float")
df["roof_shf"] = rooftop["shf"].iloc[:611].astype("float")
df = df.dropna()
X = df["shf"].values.reshape(-1, 1)  # values converts it into a numpy array
Y = df["roof_shf"].iloc[:611].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column

In [ ]:
df = pd.DataFrame()
# df = cdf.filter([""])
df["obukhov"] = cdf_unstable["obukhov"].iloc[4:].astype("float")
df["roof_ob"] = rooftop["obukhov"].iloc[:611].astype("float")
# df = df[df["obukhov"] !=0]
df = df.dropna()
X = df["obukhov"].values.reshape(-1, 1)  # values converts it into a numpy array
Y = df["roof_ob"].iloc[:611].values.reshape(-1, 1)  # -1 means that calculate the dimension of rows, but have 1 column

In [ ]:

fig = plt.figure(figsize=(8, 8))

plt.scatter(X, Y, marker=".", color="gray")
plt.title("Sensible Heat Flux Regression for Schießstand, 24$^{th}$ May 2019", fontweight="bold", fontsize = 22)
# plt.xlabel("Scintillometer Flux", fontsize = 20)
plt.ylabel("InnFLUX Sensible Heat Flux, [W$\cdot m^{-2}$]", fontsize = 20)
ax = plt.gca()
ax.set_xlabel("Scintillometer Sensible Heat Flux, [W$\cdot m^{-2}$]", fontsize = 20)

# Linear Regression

linear_regressor = LinearRegression()  # create object for the class
reg = linear_regressor.fit(X, Y)  # perform linear regression
Y_pred = linear_regressor.predict(X) # make predictions
# plt.plot(X, Y_pred, color='black')

plt.text(0.7, 0.9, str("R$^{2}$= " + str(round(reg.score(X,Y), 5))), horizontalalignment='left',
     verticalalignment='bottom', transform=ax.transAxes)

plt.show()
# 
fig.savefig('./plots/ERTEL/shf_reg_clean.png', bbox_inches='tight')

In [ ]:
hourly_cdf["obukhov_mean"] = 0
# hourly_cdf["shf_std"] = hourly_cdf["shf"].astype("float64").resample("H").std()
hourly_cdf["obukhov_mean"] = cdf["obukhov"].astype("float64").resample("600S").mean()

fig = plt.figure(figsize=(26, 6))
cdf["obukhov"].astype("float64").plot(color="grey", label="Time series")
hourly_cdf["obukhov_mean"].astype("float64").plot(
    color="black", linestyle="dashed", label="10-minute mean")

plt.legend(loc="lower left")
plt.xlabel("Time, CET")
ax = plt.gca()
ax.xaxis.set_major_formatter(
    dates.DateFormatter('%H:%M'))  # hours and minutes
plt.show()

In [ ]:
fig = plt.figure(figsize=(26, 6))
cdf = cdf.tz_convert("UTC")
# sdf["H_convection"].plot(color="black", label="Scintillometer")
cdf["shf"].plot(color="red", label="Iterated Flux")

plt.legend(loc="upper left")
plt.title(
    r"Sensible Heat Fluxes $Q_{H}$ from Scintillometer and from Iteration",
    fontweight="bold")
plt.xlabel("Time")
plt.ylabel("Sensible Heat Flux, [W$\cdot$m$^{-2}$]")
ax = plt.gca()
ax.xaxis.set_major_formatter(
    dates.DateFormatter('%H:%M'))  # hours and minutes

In [ ]:
cdf

In [ ]:
cdf_backup = cdf.copy()

In [ ]:
cdf = cdf.tz_convert("UTC")

In [ ]:
z_eff

In [ ]:
column_headers = ["yyyy", "MM", "day", "HH", "MM", "zm", "d", "z0",
                  "u_mean", "L", "sigma_v", "u_star", "wind_dir",
                  "temperature, pressure, windspeed, shf, H_free, obukhov"]
ndf = cdf.copy().filter(["wind_dir", "temperature", "pressure", "windspeed", "shf", "H_free", "obukhov"])

ndf["yyyy"] = cdf.index.year
ndf["mm"] = cdf.index.month
ndf["day"] = cdf.index.day
ndf["HH"] = cdf.index.hour
ndf["MM"] = cdf.index.minute
ndf["zm"] = 40.9
ndf["d"] = 0.7 * 12
ndf["z0"] = -999
ndf["u_mean"] = cdf["windspeed"].mean()
ndf["L"] = cdf["obukhov"].fillna(value=-999)
ndf["sigma_v"] = (np.sqrt(((cdf["windspeed"]-ndf["u_mean"])**2)).mean())
ndf["sigma_v"] = ndf["sigma_v"].fillna(value=-999)
ndf["u_star"] = cdf["u_star"].fillna(value=-999)
ndf["wind_dir"] = cdf["wind_dir"].fillna(value=-999)
ndf

In [ ]:
# ndf.to_csv(path_or_buf="../ffp/ffp_midpoint.csv", index=None) 

In [ ]:
column_headers = ["yyyy", "MM", "day", "HH", "MM", "zm", "d", "z0", "u_mean", "L", "sigma_v", "u_star", "wind_dir"]
ndf = cdf.copy().filter("wind_dir")

ndf["yyyy"] = cdf.index.year
ndf["mm"] = cdf.index.month
ndf["day"] = cdf.index.day
ndf["HH"] = cdf.index.hour
ndf["MM"] = cdf.index.minute
ndf["zm"] = 31.42
ndf["d"] = 0.7 * 12
ndf["z0"] = -999
ndf["u_mean"] = cdf["windspeed"].mean()
ndf["L"] = cdf["obukhov"].fillna(value=-999)
ndf["sigma_v"] = (np.sqrt(((cdf["windspeed"]-ndf["u_mean"])**2)).mean())
ndf["sigma_v"] = ndf["sigma_v"].fillna(value=-999)
ndf["u_star"] = cdf["u_star"].fillna(value=-999)
ndf["wind_dir"] = cdf["wind_dir"].fillna(value=-999)
ndf

In [ ]:
ndf_backup = ndf.copy()
ndf = ndf.reset_index(drop=True)
ndf

In [ ]:
# ndf.to_csv(path_or_buf="../ffp/ffp_hq1.csv", index=None)
